In [1]:
DATAPATH = '../data3'
MAGPATH  = '../../mag3'
CPPPATH  = '../cpp'

PATHS = dict(CPPPATH=CPPPATH,MAGPATH=MAGPATH,DATAPATH=DATAPATH)
print PATHS

import sys
sys.path.append('../ipynb-mag2/') # python lib path
import cuplib

import pandas as pd
import numpy as np
import commands
%pylab inline

{'CPPPATH': '../cpp', 'DATAPATH': '../data3', 'MAGPATH': '../../mag3'}
Populating the interactive namespace from numpy and matplotlib


In [2]:
# citcount_p
'''
cmd = 'g++ %(CPPPATH)s/citcount_p.cpp -o %(CPPPATH)s/citcount_p -O3 -std=c++11 && \
time %(CPPPATH)s/citcount_p \
%(MAGPATH)s/PaperReferences.txt %(DATAPATH)s/citcount_p.tsv' % PATHS
print cmd

#print commands.getoutput('%s >> %s/run.log' % (cmd,PATHS['DATAPATH']))
'''
pass

In [3]:
# citcount_a

cmd = 'g++ %(CPPPATH)s/citcount_a.cpp -o %(CPPPATH)s/citcount_a -O3 -std=c++11 && \
time %(CPPPATH)s/citcount_a \
%(DATAPATH)s/citcount_p.tsv %(MAGPATH)s/PaperAuthorAffiliations.txt %(DATAPATH)s/citcount_a.tsv' % PATHS
print cmd

print commands.getoutput('%s >> %s/run.log' % (cmd,PATHS['DATAPATH']))

g++ ../cpp/citcount_a.cpp -o ../cpp/citcount_a -O3 -std=c++11 && time ../cpp/citcount_a ../data3/citcount_p.tsv ../../mag3/PaperAuthorAffiliations.txt ../data3/citcount_a.tsv
155.49user 59.81system 3:55.62elapsed 91%CPU (0avgtext+0avgdata 15937484maxresident)k
36231832inputs+1099632outputs (2major+101455minor)pagefaults 0swaps


In [4]:
# citsum_pa

cmd = 'g++ %(CPPPATH)s/citsum_pa.cpp -o %(CPPPATH)s/citsum_pa -O3 -std=c++11 && \
time %(CPPPATH)s/citsum_pa \
%(DATAPATH)s/citcount_a.tsv %(MAGPATH)s/PaperAuthorAffiliations.txt %(DATAPATH)s/citsum_pa.tsv' % PATHS
print cmd

print commands.getoutput('%s >> %s/run.log' % (cmd,PATHS['DATAPATH']))

g++ ../cpp/citsum_pa.cpp -o ../cpp/citsum_pa -O3 -std=c++11 && time ../cpp/citsum_pa ../data3/citcount_a.tsv ../../mag3/PaperAuthorAffiliations.txt ../data3/citsum_pa.tsv
172.69user 86.95system 4:47.48elapsed 90%CPU (0avgtext+0avgdata 15643008maxresident)k
27550104inputs+1835944outputs (2major+68593minor)pagefaults 0swaps


## Pcits

In [5]:
df = pd.read_csv(DATAPATH+'/citcount_p.tsv',sep='\t',header=None,index_col=0,names='Pid Pcits'.split(),nrows=None)
df

,Pcits
Pid,
100,34
104,1
106,2
125,1
136,42
245,36
285,6
290,18
372,9


In [6]:
field = 'Pcits'
cuplib.write_submission(df[field],DATAPATH+'/final/%s/'%field,hexids=False) # TODO

'../data3/final/Pcits/results.tsv'

## Ppr

In [7]:
cmd = 'g++ %(CPPPATH)s/randomwalk/pagerank.cpp -o %(CPPPATH)s/randomwalk/pagerank -O3 -std=c++11 && \
time %(CPPPATH)s/randomwalk/pagerank %(DATAPATH)s/PaperReferences.inlinks %(DATAPATH)s/PaperReferences.pr' % PATHS

print cmd

print commands.getoutput('%s >> %s/run.log' % (cmd,PATHS['DATAPATH']))

g++ ../cpp/randomwalk/pagerank.cpp -o ../cpp/randomwalk/pagerank -O3 -std=c++11 && time ../cpp/randomwalk/pagerank ../data3/PaperReferences.inlinks ../data3/PaperReferences.pr
2411.60user 38.14system 40:53.35elapsed 99%CPU (0avgtext+0avgdata 8098472maxresident)k
1423984inputs+2024528outputs (1major+57330511minor)pagefaults 0swaps


In [8]:
%%time
dfpr = pd.read_csv(DATAPATH+'/PaperReferences.pr',sep='\t',header=None,index_col=0,names='Inc Ppr'.split(),
                   nrows=None)
index = pd.read_csv(DATAPATH+'/PaperReferences.nodes',sep='\t',header=None,names='Pid'.split(),
                    nrows=dfpr.shape[0])['Pid']#.map(cuplib.hex2int)
dfpr.set_index(index,inplace=True)

CPU times: user 20.8 s, sys: 1.15 s, total: 21.9 s
Wall time: 23.1 s


In [9]:
field = 'Ppr'
cuplib.write_submission(dfpr[field],DATAPATH+'/final/%s/'%field,hexids=False) # TODO
#2m01s

'../data3/final/Ppr/results.tsv'

In [10]:
# dfpr.sort('Ppr',ascending=False)

## Ties

In [17]:
df = pd.read_csv(DATAPATH+'/final/sRCR11/results.tsv', # TODO
                 sep='\t',header=None,names='Pid sRCR'.split(),index_col=0,nrows=None)
df.set_index(df.index.map(cuplib.hex2int),inplace=True)

In [18]:
df = df.join(
    pd.read_csv(DATAPATH+'/citsum_pa.tsv',sep='\t',index_col=0,header=None,nrows=None,names=['PAcits']),
    how='outer')

In [26]:
df.fillna(0,inplace=True)
df

,sRCR,PAcits
20,0.001343,6
23,0.000000,21
37,0.000000,11
58,0.000000,10
100,0.000130,4142
104,0.000076,15
106,0.000013,2
108,0.001343,0
125,0.000011,80
136,0.000093,324


In [27]:
from collections import Counter
counter = Counter(df.sRCR)
counter.most_common(5)

[(0.0, 17655976),
 (0.00134253, 9485769),
 (0.00067126300000000007, 2332),
 (0.00044750900000000001, 1914),
 (0.00268505, 1768)]

In [28]:
df['sRCR_PAcits'] = cuplib.breakties(df.sRCR,df.PAcits)
df

,sRCR,PAcits,sRCR_PAcits
20,0.001343,6,1.342530e-03
23,0.000000,21,2.226219e-16
37,0.000000,11,1.113110e-16
58,0.000000,10,1.001799e-16
100,0.000130,4142,1.297511e-04
104,0.000076,15,7.579810e-05
106,0.000013,2,1.335290e-05
108,0.001343,0,1.342530e-03
125,0.000011,80,1.070950e-05
136,0.000093,324,9.338950e-05
